## AWS

### Instances on AWS

To use AWS you will need to set up a public/private key. There are two things to keep track of:
* the <code>.pem</code> file containing an encryption key used when shelling in to AWS;
* the AWS_ACCESS_KEY_ID/AWS_SECRET_ACCESS_KEY that are provided when setting up security.

Follow these instructions for [getting the AWS command line interface working](http://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-started.html "AWS command line interface")

You can shell in using:  
<pre>
ssh -X -i /path/to/myAWSKey.pem user@ec2-54-208-202-XXX.compute-1.amazonaws.com
</pre>

*It might be necessary to omit the* <code>-i</code> *when initiating* <code>Xvfb</code>, *<a href="#Xvfb">as described below</a>.*

Similarly, you can copy files there using <code>scp</code>:
<pre>
scp -i /path/to/myAWSKey.pem &lt;local_file_list&gt; user@ec2-54-208-202-XXX.compute-1.amazonaws.com:
</pre>

### Caffe on AWS g2.2xlarge instance

A GPU-enabled AWS instance is a requirement for Caffe, if speed is important. Since my application allows users to supply images interactively, I've gone that route.  
  
Two posts are especially useful for getting Caffe working on Fedora, and working on AWS:
* [Thoughts on Cognition: Caffe on AWS with Fedora 20, CUDA 6.5 and CUDNN](http://cjbeyer.blogspot.com/2014/10/caffe-on-aws-with-fedora-20-cuda-65-and.html "A blog posting by Charles Beyer")
* [Caffe on EC2 Ubuntu 14.04 Cuda 7](https://github.com/BVLC/caffe/wiki/Caffe-on-EC2-Ubuntu-14.04-Cuda-7 "Caffe's own Ubuntu AMI")

The first of these gives clear guidance on how to build Caffe on a Fedora-based system (what I have on my notebook). The only thing missing is that neither the Caffe installation documentation, nor this blog post, explicitly tell of the need to install <code>openblas-devel</code> if using <code>openblas</code>. The error messages that result when that is missing did not give me *the slightest indication* of the true nature of the problem.  
  
I was able to use the pre-built AMI from Caffe without trouble. Note that the username for an Ubuntu instance is <code>ubuntu</code>, rather than <code>user</code>. (This is relevant for using <code>ssh/scp</code> &mdash; see above.)  
  
#### <a name="Xvfb">Xvfb</a>

Running my app requires X, which is problematic when logging out of the AWS instance. <code>Xvfb</code> is the fix. While logged onto AWS, type:

<pre>
apt-get install xorg-x11-server-Xvfb
Xvfb :100 -ac
export DISPLAY=:100.0
</pre>

[See Running X applications headless using Xvfb](http://blog.kagesenshi.org/2007/06/running-x-applications-headless-using.html "a blog entry by Izhar Firdaus") for more details.

#### tmux

To prevent the application from quitting when logging off of the AWS instance, it is neccesary to run it in background <code>&lt;ctrl-b&gt;</code> and detach it <code>&lt;D&gt;</code>. This can be done with <code>tmux</code>:

<pre>
tmux new -s Flask
sudo python app/my_app.py
&lt;ctrl-b&gt;
&lt;D&gt;
</pre>

To verify a running process:
<pre>
tmux ls
</pre>

To be tested:  
  
On a local machine, if <code>flask</code> application code is modified, the application automatically reloads the software to affect the change.
It may work on AWS to re-attach to <code>tmux</code>, insert new code, and then detach again.
Supposing that the new version of the code is sitting in <code>~/</code>, while the application resides in <code>app/</code>.
Try something like this:

<pre>
tmux attach -t Flask
sudo mv -i my_app.py app/
&lt;D&gt;
</pre>